# Introduction

Creating a mutual information stainer in Python

In [13]:
import pandas as pd
import random
import numpy as np
# from scipy.optimize import basinhopping
# from sklearn.metrics import mutual_info_score
from tqdm.notebook import tqdm_notebook
import time

In [2]:
def get_row_col_sums(x):
    """
    Get the row and column sums respectively. Where X is a 2d numpy array
    """
    return (x.sum(axis=1), x.sum(axis=0))

# Computing Mutual Information

Using __Shannon entropy and the mutual information of two variables__ as implemented in `DescTools` in R. 

There is no Python version of this package (that I could find at any rate) so implementing it myself. 

Cross-checked with the R version. Results are identical. 

In [3]:
def entropy_pair(table):
    total=table.sum()
    probs = []
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            probs.append(table[i,j]/total)
    return np.sum([-p * np.log2(p) for p in probs if p > 0])
#     for c1 in set(X):
#         for c2 in set(Y):
#             probs.append(np.mean(np.logical_and(X == c1, Y == c2)))

#     return np.sum([-p * np.log2(p) for p in probs if p > 0])

def entropy(x):
    """
    Not in use
    """
    probs = [np.mean(x == c) for c in set(x)]
    return np.sum([-p * np.log2(p) for p in probs])

def MutInf(table):
    """
    Computes the mutual information using Shannon entropy & mutual information. 
    Input is a contingency table
    
    """
    # Get the entropy of the marginals
    rowsums, colsums = get_row_col_sums(table)
    ent_x = sum([-i * np.log2(i) for i in (rowsums/rowsums.sum()) if i > 0])
    ent_y = sum([-i * np.log2(i) for i in (colsums/colsums.sum()) if i > 0])
    # Get mutual information
    return ent_x+ent_y-entropy_pair(table)

In [4]:
x = np.array(["a","b","a","c","b","c"])
y = np.array(["b","a","a","c","c","b"])



[simulated annealing](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html?highlight=basinhopping)

[guide](https://machinelearningmastery.com/simulated-annealing-from-scratch-in-python/#:~:text=Simulated%20Annealing%20is%20a%20stochastic,algorithms%20do%20not%20operate%20well.)

In [5]:
x_vec = [14, 22, 7, 12, 19, 11, 9, 18, 21, 14, 5, 11]
# Python version of table() in R
x=np.array(x_vec).reshape(-1, 3).transpose()
x

array([[14, 12,  9, 14],
       [22, 19, 18,  5],
       [ 7, 11, 21, 11]])

In [6]:
def gen_number(x):
    table=x.copy()
    nrows, ncols=table.shape
    RR=random.sample(range(0, nrows), 2)
    CC=random.sample(range(0, ncols), 2)
    org_values=[table[RR[0], CC[0]], table[RR[1], CC[1]]]
    
    while org_values[0] < 1 or org_values[1]<1:
        RR=random.sample(range(0, nrows), 2)
        CC=random.sample(range(0, ncols), 2)
        org_values=[table[RR[0], CC[0]], table[RR[1], CC[1]]]
        
    table[RR[0], CC[0]] = table[RR[0], CC[0]]-1
    table[RR[1], CC[1]] = table[RR[1], CC[1]]-1
    
    table[RR[0], CC[1]] = table[RR[0], CC[1]]+1
    table[RR[1], CC[0]] = table[RR[1], CC[0]]+1
    
    return table 

In [7]:
x

array([[14, 12,  9, 14],
       [22, 19, 18,  5],
       [ 7, 11, 21, 11]])

In [8]:
q = gen_number(x)
q

array([[13, 13,  9, 14],
       [22, 19, 18,  5],
       [ 8, 10, 21, 11]])

In [9]:
print(get_row_col_sums(x))
print(get_row_col_sums(q))

(array([49, 64, 50]), array([43, 42, 48, 30]))
(array([49, 64, 50]), array([43, 42, 48, 30]))


The `gen_nbr()` function to produce candidate contingency tables works as intended. 

# Simulated Annealing 

In [10]:
def simul_anneal(initital_table, obj=MutInf, gen_fn=gen_number, n=500, temp=10):
    # Copy of initial point
    best = initital_table.copy()
    # Evaluate Initial point
    best_eval = obj(best)
    # CUrrent Working Solution
    curr, curr_eval = best, best_eval
    # Run the algorithm
    for i in range(n):
        # Take a step
        cand = gen_fn(curr)
        # Evaluate new candidate point
        cand_eval = obj(cand)
        # Check for new best solution
        if cand_eval > best_eval:
            best, best_eval = cand, cand_eval
        # Difference between candidate and current point evaluation
        diff=cand_eval-curr_eval
        # calculate temperature
        t = temp/float(i+1)
        # Metropolis acceptance criterion
        metropolis=np.exp(diff/t)
        if diff > 0 or random.uniform(0,1) < metropolis:
            curr, curr_eval = cand, cand_eval
    return best, best_eval
        
        

In [25]:
# Mean Mutual Information when run WITH metropolis creterion
metro = [] 
for i in tqdm_notebook(range(1000)):
    metro.append(simul_anneal(x)[1])
metro=np.mean(metro)
print(metro)

  0%|          | 0/1000 [00:00<?, ?it/s]

1.1824340899891843


In [23]:
# Mean Mutual Information when run WITHOUT metropolis creterion
no_metro = [] 
for i in tqdm_notebook(range(1000)):
    no_metro.append(simul_anneal(x)[1])
no_metro=np.mean(no_metro)
print(no_metro)

  0%|          | 0/1000 [00:00<?, ?it/s]

1.0551288343275393


In [115]:
test = pd.DataFrame({"Passes Final" : (['Yes', 'No']*20),
             "Midterm Grade" : ((['D']*10)+(['C']*10)+(['B']*10)+(['A']*10))})
test.head(2)

,Passes Final,Midterm Grade
0,Yes,D
1,No,D


In [106]:
df=test.copy()

# Record label names
y_name, x_name = df.columns
# Get contingency table 
table = pd.crosstab(df[y_name],
       df[x_name], 
       margins=False)
x_index = table.index
y_index = table.columns
# Convert table to numpy array
table=table.to_numpy()
# Record old row & column sums
rowsum_old, colsum_old = get_row_col_sums(table)

#Perform simulated annealing
table, new_mut=simul_anneal(table)
# New row & columns sums
rowsum_new, colsum_new = get_row_col_sums(table)

if not (np.array_equal(rowsum_old, rowsum_new) and np.array_equal(colsum_old, colsum_new)):
    raise Exception("Marginals not the same")

# Convert to dataframe to return
df = pd.DataFrame(table)
# Reset field names
df.index=x_index
df.columns=y_index

# Unpivot frequency table

df = df.stack()
df = df.index.repeat(df).to_frame().reset_index(drop=True)

In [111]:
def new_mutual_cols(df):
    
    df = df.copy()
    
    # Record label names
    y_name, x_name = df.columns
    # Get contingency table 
    table = pd.crosstab(df[y_name],
           df[x_name], 
           margins=False)
    x_index = table.index
    y_index = table.columns
    # Convert table to numpy array
    table=table.to_numpy()
    old_mut = MutInf(table)
    # Record old row & column sums
    rowsum_old, colsum_old = get_row_col_sums(table)

    #Perform simulated annealing
    table, new_mut=simul_anneal(table)
    # New row & columns sums
    rowsum_new, colsum_new = get_row_col_sums(table)

    if not (np.array_equal(rowsum_old, rowsum_new) and np.array_equal(colsum_old, colsum_new)):
        raise Exception("Marginals not the same")

    # Convert to dataframe to return
    df = pd.DataFrame(table)
    # Reset field names
    df.index=x_index
    df.columns=y_index

    # Unpivot frequency table
    df = df.stack()
    df = df.index.repeat(df).to_frame().reset_index(drop=True)
    
    # Return Dataframe
    return (df, old_mut, new_mut)

In [114]:
new_mutual_cols(test)

,Passes Final,Midterm Grade
0,No,C
1,No,C
2,No,C
3,No,C
4,No,C
5,No,C
6,No,C
7,No,C
8,No,C
9,No,C
